In [ ]:
from espressomd import System, interactions, electrostatics
from espressomd.io.writer import vtf

import numpy as np
from scipy import optimize
%matplotlib inline
import matplotlib.pyplot as plt


Theory: this notebook introduces electrostatics methods (it will replace the current tutorial for charged systems). The physical goal is to study counterion condensation around a charged rod (simple cell model for a polyelectrolyte) and recreate some of the results from "Deserno, Markus, Christian Holm, and Sylvio May. "Fraction of condensed counterions around a charged rod: Comparison of Poisson− Boltzmann theory and computer simulations." Macromolecules 33.1 (2000): 199-206."

In [ ]:

# system parameters
rod_length = 24.

# number of beads that make up the rod
N_rod_beads = 50

bjerrum_length = 1.0
kT = 1.





In [ ]:
system = System(box_l=3*[rod_length])
system.time_step = 0.01
system.cell_system.skin = 0.4

In [ ]:
wca_epsilon = 1.0
ion_diameter = 1.0
rod_radius = 1.0

# particle types 
rod_type = 1
counterion_type = 2

#ion-ion interaction
system.non_bonded_inter[counterion_type,counterion_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter)

# ion-rod interaction
system.non_bonded_inter[counterion_type,rod_type].wca.set_params(
      epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)

In [ ]:
def setup_rod_and_ions(system, ion_valency, counterion_type,
                 rod_line_charge, N_rod_beads, rod_type):
    
    # calculate charge of the single rod beads
    total_rod_charge = rod_charge_dens*system.box_l[0]
    rod_charge_per_bead = total_rod_charge/N_rod_beads
    
    # number of counterions
    N_ions = int(total_rod_charge/ion_valency)
    
    assert(abs((total_rod_charge-N_ions*ion_valency)/total_rod_charge)<1e-8)
    
    for idx in range(N_rod_beads):
        system.part.add(pos=[rod_length/2.,rod_length/2.,idx/N_rod_beads], 
                        type=rod_type, q=rod_charge_per_bead, fix=3*[True])

    for _ in range(N_ions):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=counterion_type, q=-ion_valency)
    

In [ ]:
counterion_valency = 2
rod_charge_dens = 1



setup_rod_and_ions(system, counterion_valency, counterion_type,
             rod_charge_dens, N_rod_beads, rod_type)

In [ ]:
p3m_params = {'prefactor':kT*bjerrum_length,
              'accuracy':1e-3}
p3m = electrostatics.P3M(**p3m_params)
system.actors.add(p3m)

In [ ]:
def remove_overlap(system, sd_params):   
    #remove overlap by steepest descent
    # Set up steepest descent integration
    system.integrator.set_steepest_descent(f_max=sd_params['f_max'],
                                           gamma=sd_params['damping'],
                                           max_displacement=sd_params['max_displacement'])
    
    # Initialize integrator to obtain initial forces
    system.integrator.run(0)
    maxforce = np.max(np.linalg.norm(system.part[:].f))
    energy = system.analysis.energy()['total']
    
    i = 0
    while i < sd_params['max_steps']//sd_params['emstep']:
        prev_maxforce = maxforce
        prev_energy = energy
        print(prev_energy)
        system.integrator.run(sd_params['emstep'])
        maxforce = np.max(np.linalg.norm(system.part[:].f))
        relforce = np.abs((maxforce-prev_maxforce)/prev_maxforce)
        energy = system.analysis.energy()['total']
        relener = np.abs((energy-prev_energy)/prev_energy)
        print("minimization step: {:4.0f}\tmax. rel. force change:{:+3.3e}\trel. energy change:{:+3.3e}".format((i+1)*sd_params['emstep'],relforce, relener))
        if relforce < sd_params['f_tol'] or relener < sd_params['e_tol']:
            break
        i += 1
        
    system.integrator.set_vv()

In [ ]:
#steepest descent params
STEEPEST_DESCENT_PARAMS = {'f_tol':1e-2,
                          'e_tol':1e-5,
                          'f_max':0,
                          'damping':30,
                          'max_steps':10000,
                          'max_displacement':0.01,
                          'emstep':10}

remove_overlap(system,STEEPEST_DESCENT_PARAMS)

In [ ]:
langevin_params = {'kT':kT,
                   'gamma':0.5,
                   'seed':42}
system.thermostat.set_langevin(**langevin_params)

In [ ]:
def integrate_calc_observables(system, N_frames, steps_per_frame, ion_types):
    energies = []
    radial_distances = []
    
    particles_by_type = {}
    radial_distances = {}
    for ion_type in ion_types:
        particles_by_type[ion_type] = system.part.select(type=ion_type)
        radial_distances[ion_type] = []
    
    system_center = np.array(system.box_l)/2.
    
    for _ in range(N_frames):
        # run run the simulation for a few steps
        system.integrator.run(steps_per_frame)

        energies.append(system.analysis.energy()['total'])
        
        for ion_type, ions in particles_by_type.items():
            for ion in ions:
                radial_distances[ion_type].append(np.linalg.norm(ion.pos_folded[0:2]-system_center[0:2]))
    
    return energies, radial_distances




In [ ]:
# run and look at energies to see how long it takes until equilibration and what the fluctuation timescale is

# MD frames to go
N_frames = 100

# number of timesteps per frame
steps_per_frame = 100
energies, distances = integrate_calc_observables(system, N_frames, 
                                       steps_per_frame, [counterion_type])

In [ ]:
def clear_system(system):
    system.thermostat.turn_off()
    system.part.clear()
    system.time = 0.
    

In [ ]:
clear_system(system)

In [ ]:
fig1 = plt.figure()
#todo plot vs time step so steps_per_frame can be tuned
plt.plot(energies)
plt.xlabel('time frames')
plt.ylabel('system total energy')

In [ ]:
#use the plot to determine the warmup time and the appropriate number of steps per frame
warmup_steps = 100

N_frames = 200
steps_per_frame = 100#*200

In [ ]:
#TODO replace this function by more intuitive behaviour of p3m.tune()
def retune_p3m(p3m):
    #reset p3m tunable parameters to default to force a retune
    p3m_default_params = p3m.default_params()
    p3m.tune(mesh = p3m_default_params['mesh'],
             cao = p3m_default_params['cao'],
             r_cut = 0,
             #TODO change default value in interface
             alpha = p3m_default_params['alpha'])

In [ ]:
#run the system with different parameter sets but same manning parameter
runs = [{'params':{'counterion_valency':2, 'rod_charge_dens':1},
         'distances':None},
        {'params':{'counterion_valency':1, 'rod_charge_dens':2},
         'distances':None}
       ]

for run in runs:
    setup_rod_and_ions(system, run['params']['counterion_valency'], counterion_type,
                 run['params']['rod_charge_dens'], N_rod_beads, rod_type)
    retune_p3m(p3m)
    remove_overlap(system, STEEPEST_DESCENT_PARAMS)
    system.thermostat.set_langevin(**langevin_params)
    
    energies, distances = integrate_calc_observables(system, N_frames, 
                                       steps_per_frame, [counterion_type])
    clear_system(system)
    run['distances'] = distances[counterion_type]
    print('simulation for parameters {} done\n'.format(run['params']))

In [ ]:
def calc_cum_hist(values, bins):
    hist, bins = np.histogram(values, bins=log_bins)
    cum_hist = np.cumsum(hist)
    return cum_hist/cum_hist[-1]

In [ ]:
log_bins=np.logspace(np.log10(rod_radius), np.log10(rod_length),num=30)

fig, ax = plt.subplots()
for run in runs:
    cum_hist = calc_cum_hist(run['distances'], log_bins)
    ax.plot(log_bins[1:],cum_hist, label = 'lambda = {}, nu = {}'.format(run['params']['rod_charge_dens'], 
                                                                          run['params']['counterion_valency']))
    
ax.set_xscale('log')
ax.legend()
plt.xlabel('r')
plt.ylabel('P(r)')

In [ ]:
def eq_to_solve_for_gamma(gamma, manning_parameter, rod_radius, rod_length):
    #eq 7 - eq 6 from 10.1021/ma990897o
    return gamma*np.log(rod_length/rod_radius) - np.arctan(1/gamma) + np.arctan((1-manning_parameter)/gamma)

def calc_manning_radius(gamma,rod_length):
    #eq 7 from 10.1021/ma990897o
    return rod_length*np.exp(-np.arctan(1./gamma)/gamma)  

def calc_PB_probability(r, manning_parameter, gamma, manning_radius):
    #eq 8 and 9 from 10.1021/ma990897o
    return 1./manning_parameter + gamma/manning_parameter * np.tan(gamma*np.log(r/manning_radius))


In [ ]:
rod_charge_density = 2
ion_valency = 1
manning_parameter_times_valency = bjerrum_length*rod_charge_density*ion_valency
#for multivalent ions, the manning parameter xi has to be multiplied by the valency.
#the result therefore depends only on the product of rod_charge_dens and ion_velancy, so we only need one curve

gamma = optimize.fsolve(eq_to_solve_for_gamma, 1, args = (manning_parameter_times_valency,rod_radius,rod_length))
manning_radius = calc_manning_radius(gamma, rod_length)

PB_probability = calc_PB_probability(log_bins, manning_parameter_times_valency,gamma, manning_radius)


ax.plot(log_bins, PB_probability, label = f'PB xi*nu = {manning_parameter_times_valency}')
ax.legend()
fig

In [ ]:
# overcharging through added salt

In [ ]:
def add_salt(system, anion_params, cation_params):        
    for _ in range(anion_params['number']):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=anion_params['type'], q=-anion_params['valency'])
        
    for _ in range(cation_params['number']):
        system.part.add(pos=np.random.random(3) * system.box_l, 
                        type=cation_params['type'], q=cation_params['valency'])
    

In [ ]:
anion_params = {'type':3,
                'valency':1,
                'number':10}
cation_params = {'type':4,
                 'valency':1,
                 'number':10}

total_anion_charge = -anion_params['number']*anion_params['valency']
total_cation_charge = cation_params['number']*cation_params['valency']
assert( abs(total_anion_charge+ total_cation_charge)/total_cation_charge < 1e-10)

counterion_valency = 1
rod_charge_dens = 2

all_ion_types = [counterion_type, anion_params['type'],cation_params['type'] ]

#set interactions of salt with the rod and all ions
for salt_type in [anion_params['type'], cation_params['type']]:
    system.non_bonded_inter[salt_type,rod_type].wca.set_params(
          epsilon=wca_epsilon, sigma=ion_diameter/2. + rod_radius)
    for ion_type in all_ion_types:
        system.non_bonded_inter[salt_type,ion_type].wca.set_params(
          epsilon=wca_epsilon, sigma=ion_diameter)
    



In [ ]:
clear_system(system)
setup_rod_and_ions(system, counterion_valency, counterion_type,
                   rod_charge_dens, N_rod_beads, rod_type)
add_salt(system, anion_params, cation_params)
retune_p3m(p3m)
remove_overlap(system, STEEPEST_DESCENT_PARAMS)
system.thermostat.set_langevin(**langevin_params)

energies, distances = integrate_calc_observables(system, N_frames, 
                                   steps_per_frame, all_ion_types)


In [ ]:
counterion_hist = calc_cum_hist(distances[counterion_type], log_bins)
anion_hist = calc_cum_hist(distances[anion_params['type']], log_bins)
cation_hist = calc_cum_hist(distances[cation_params['type']], log_bins)

charge_hist = cation_params['valency'] * cation_hist - anion_params['valency'] * anion_hist - counterion_valency * counterion_hist

fig2, ax2 = plt.subplots()
#flip the histogram (couterions are negative, but the plot looks nicer pn the positive axis)
ax2.plot(log_bins[1:], -charge_hist)
ax2.set_xscale('log')
plt.xlabel('r')
plt.ylabel('P(r)')

